Copyright 2021 DeepMind Technologies Limited

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# Arnheim 3 - Collage

**Piotr Mirowski, Dylan Banarse, Mateusz Malinowski, Yotam Doron, Oriol Vinyals, Simon Osindero, Chrisantha Fernando**

DeepMind, 2021

![picture](https://github.com/deepmind/arnheim/raw/main/images/arnheim3_examples.png)
Clockwise from top left: "Sri Lankan objects" (200 transparent patches); "Waves" (70 masked transparnecy patches with background); "Fruit bowl" (100 opacity patches); "Fruit bowl" (100 transparent patches); "Face" (7 opacity patches); "Swans" (100 masked transparent patches); "Chicken" (70 masked transparent patches); "Dancer" (40 transparent patches). See description in the [videos](https://www.youtube.com/watch?v=HKDQsrO5xF4&list=PLKhLdFXp1JN5SEV56w9OWWsT5pAz9z7G_) for settings.

**STEPS:**

Quickstart:
1. Click "Connect" in the top right corner
1. Runtime -> Run all

Play around:
* Parameters section to configure collage creation settings
* Image patches section to upload your own patches
* Make Drawing section to set the text prompt

**An Exploration of Architectures and Losses for Painting and Drawing**

Arnheim 3 is an algorithm which generates collages by training by gradient descent a network which applies affine transformations, i.e translation, scaling, rotation, and shear to a set of image patches, this set being subject to evolution in the outer loop. 

The signal for how good an image is comes from CLIP, a text-image dual encoder. This work simplifies and extends Arnheim 2 which also used CLIP but generated SVG strokes using a more complex hierarchical stroke grammar. 

Here you can experiment with a variety of rendering methods for combining patches in a learnable way.

**New Features**
1. Compositional Images

  Uses 3x3 prompts covering over-lapping regions of the image to specify different content across the whole image. The main prompt guides the direction of overall image.

1. Coloured Background

  User-selectable background colour or use of uploaded images.

1. Interactive Patch Placement

  Stop the "Create collage! (Loop)" cell at any time and run the "Tinker with patches" cell below it to adjust individual patches with sliders. Then re-run the "Create collage! (Loop)" cell to continue generation.

**Tips**

**opacity rendering** uses alpha and depth to render semi-opaque overlapping patches which allow gradients to be used during learning. The translucency is reduced over the course of learning to end with opaque patches. When using a small number of patches evolution can perform better than learning alone. For example, with only 7 patches a population of 10 with the Evolutionary Strategies method applied at every step can yield good results. The settings to get the face image above were:

* Prompt “Face”
* 7 patches
* opacity
* 400 steps
* ES evolution every step
* POP_SIZE=10
* ROT_POS_MUTATION=0.05
* SCALE_MUTATION = 0.02
* PATCH_MUTATION = 0.2

**Transparency rendering** works well as gradients are more effective. Note that colours are additive so setting INITIAL_MIN_RGB=0.1 and INITIAL_MAX_RGB=0.5 helps reduce bleaching. Something to try is:

* 80 Patches
* Transparency
* INITIAL_MIN_RGB=0.1; INITIAL_MAX_RGB=0.5
* 15000 steps
* Microbial GA every 100 steps
* POP_SIZE = 2
* LEARNING_RATE = 0.07
* PATCH_MUTATION_PROBABILITY = 1
* Prompt "Swans on a pond"

**Masked Transparency rendering** also works well as gradients are more effective.  Something to try is:

* 200 Patches
* MASKED Transparency
* INITIAL_MIN_RGB=0.7; INITIAL_MAX_RGB=1.0
* 15000 steps
* Microbial GA every 100 steps
* POP_SIZE = 2
* LEARNING_RATE = 0.07
* PATCH_MUTATION_PROBABILITY = 1
* Prompt "Swans on a pond"

**Note that the Colab can easily run out of memory with large populations, many patches and large patch sizes! If you start to encounter CUDA memory issues try lowering the number of patches and restarting the Colab.**

# Preliminaries

In [ ]:
#@title Installation of libraries {vertical-output: true}

!nvidia-smi -L

import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
  torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
  torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
  torch_version_suffix = ""
else:
  torch_version_suffix = "+cu110"

%cd /content/
!pip install cssutils
!pip install torch-tools
!pip install   visdom
!pip install kornia
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git --no-deps

# Imports and libraries

In [ ]:
#@title Imports {vertical-output: true}
import clip
import copy
import cv2
from google.colab import drive
from google.colab import files
from google.colab.patches import cv2_imshow
import io
from kornia.color import hsv
from matplotlib import pyplot as plt
import moviepy.editor as mvp
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
import numpy as np
import os
import pathlib
import random
import requests
from skimage.transform import resize
import time
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

os.environ["FFMPEG_BINARY"] = "ffmpeg"
print("Torch version:", torch.__version__)

In [ ]:
#@title Initialise and load CLIP model {vertical-output: true}

device = torch.device("cuda")
CLIP_MODEL = "ViT-B/32"
print(f"Downloading CLIP model {CLIP_MODEL}...")
clip_model, _ = clip.load(CLIP_MODEL, device, jit=False)
!nvidia-smi

# Parameters

In [ ]:
#@title Collage configuration
#@markdown Render methods

#@markdown 
#@markdown **opacity** patches overlay each other using a combination of alpha and depth,
#@markdown **transparency** _adds_ patch colours (black therefore appearing transparent),
#@markdown and **masked transparency** blends patches using the alpha channel.
RENDER_METHOD = "masked_transparency"  #@param ["opacity", "transparency", "masked_transparency"]
NUM_PATCHES =      100#@param {type:"integer"}
COLOUR_TRANSFORMATIONS = "RGB space"  #@param ["none", "RGB space", "HSV space"]
#@markdown Invert image colours to have a white background?
INVERT_COLOURS = False #@param {type:"boolean"}

CANVAS_WIDTH = 224
CANVAS_HEIGHT = 224
MULTIPLIER_BIG_IMAGE = 4

#@markdown Use additional prompts for different regions
COMPOSITIONAL_IMAGE = True #@param {type:"boolean"}
if COMPOSITIONAL_IMAGE:
  CANVAS_WIDTH *= 2
  CANVAS_HEIGHT *= 2
  MULTIPLIER_BIG_IMAGE = int(MULTIPLIER_BIG_IMAGE / 2)

PATCH_WIDTH_MIN = 16  
PATCH_HEIGHT_MIN = 16 
PATCH_MAX_PROPORTION =  3


In [ ]:
#@title Affine transform settings

#@markdown Translation bounds for X and Y.
MIN_TRANS = -1  #@param{type:"slider", min:-1.0, max:1.0, step:0.01}
MAX_TRANS = 1  #@param{type:"slider", min:-1.0, max:1.0, step:0.01}
#@markdown Scale bounds (> 1 means zoom out and < 1 means zoom in).
MIN_SCALE =   0.5#@param {type:"number"}
MAX_SCALE =   2.0#@param {type:"number"
#@markdown Bounds on ratio between X and Y scale (default 1).
MIN_SQUEEZE =   0.5#@param {type:"number"}
MAX_SQUEEZE =   2.0#@param {type:"number"}
#@markdown Shear deformation bounds (default 0)
MIN_SHEAR = -0.2  #@param{type:"slider", min:-1.0, max:1.0, step:0.01}
MAX_SHEAR = 0.2  #@param{type:"slider", min:-1.0, max:1.0, step:0.01}
#@markdown Rotation bounds.
MIN_ROT_DEG = -180 #@param{type:"slider", min:-180, max:180, step:1}
MAX_ROT_DEG = 180 #@param{type:"slider", min:-180, max:180, step:1}
MIN_ROT = MIN_ROT_DEG * np.pi / 180.0
MAX_ROT = MAX_ROT_DEG * np.pi / 180.0


In [ ]:
#@title Colour transform settings

#@markdown RGB
MIN_RGB = -0.45  #@param {type:"slider", min: -1, max: 1, step: 0.01}
MAX_RGB = 1.0  #@param {type:"slider", min: 0, max: 1, step: 0.01}
INITIAL_MIN_RGB = 0.05  #@param {type:"slider", min: 0, max: 1, step: 0.01}
INITIAL_MAX_RGB = 0.4  #@param {type:"slider", min: 0, max: 1, step: 0.01}
#@markdown HSV
MIN_HUE = 0.  #@param {type:"slider", min: 0, max: 1, step: 0.01}
MAX_HUE_DEG = 360 #@param {type:"slider", min: 0, max: 360, step: 1}
MAX_HUE = MAX_HUE_DEG * np.pi / 180.0
MIN_SAT = 0.  #@param {type:"slider", min: 0, max: 1, step: 0.01}
MAX_SAT = 1.  #@param {type:"slider", min: 0, max: 1, step: 0.01}
MIN_VAL = 0.  #@param {type:"slider", min: 0, max: 1, step: 0.01}
MAX_VAL = 1.  #@param {type:"slider", min: 0, max: 1, step: 0.01}

In [ ]:
#@title Training settings

# Reasonable defaults:
# OPTIM_STEP = 10000 to 20000
# LEARNING_RATE = 0.1
# NUM_AUGS = 2 to 4
# GRADIENT_CLIPPING = 10.0
# USE_NORMALIZED_CLIP = True

#@markdown Number of training steps
OPTIM_STEPS = 10000    #@param{type:"slider", min:200, max:20000, step:100}

LEARNING_RATE = 0.1    #@param{type:"slider", min:0.0, max:0.3, step:0.001}
#@markdown Number of augmentations to use in evaluation
USE_IMAGE_AUGMENTATIONS = True #@param{type:"boolean"}
NUM_AUGS = 4  #@param {type:"integer"}

#@markdown Normalize colours for CLIP, generally leave this as True
USE_NORMALIZED_CLIP = False  #@param {type:"boolean"}

#@markdown Gradient clipping during optimisation
GRADIENT_CLIPPING = 10.0  #@param {type:"number"}

#@markdown Initial random search size (1 means no search)
INITIAL_SEARCH_SIZE = 1 #@param {type:"slider", min:1, max:50, step:1}

if COMPOSITIONAL_IMAGE:
  print("Using ONE image augmentations for compositional image creation.")
  USE_IMAGE_AUGMENTATIONS = True
  NUM_AUGS = 1

In [ ]:
#@title Evolution settings

# Reasonable defaults:
# POP_SIZE = 2
# EVOLUTION_FREQUENCY = 100
# MUTION SCALES = ~0.1
# MAX_MULTIPLE_VISUALISATIONS = 7

#@markdown For evolution set POP_SIZE greater than 1
POP_SIZE =    2  #@param{type:"slider", min:1, max:100}
EVOLUTION_FREQUENCY =  100#@param {type:"integer"}

#@markdown ### Genetic algorithm methods

#@markdown **Microbial** - loser of randomly selected pair is replaced by mutated winner. A low selection pressure.

#@markdown **Evolutionary Strategies** - mutantions of the best individual replace the rest of the population. Much higher selection pressure than Microbial GA.
GA_METHOD = "Microbial"  #@param ["Evolutionary Strategies", "Microbial"]
#@markdown ### Mutation levels
#@markdown Scale mutation applied to position and rotation, scale, distortio, colour and patch swaps.
POS_AND_ROT_MUTATION_SCALE = 0  #@param{type:"slider", min:0.0, max:0.3, step:0.01}
SCALE_MUTATION_SCALE = 0  #@param{type:"slider", min:0.0, max:0.3, step:0.01}
DISTORT_MUTATION_SCALE = 0  #@param{type:"slider", min:0.0, max:0.3, step:0.01}
COLOUR_MUTATION_SCALE = 0  #@param{type:"slider", min:0.0, max:0.3, step:0.01}
PATCH_MUTATION_PROBABILITY = 1  #@param{type:"slider", min:0.0, max:1.0, step:0.1}
#@markdown Limit the number of individuals shown during training
MAX_MULTIPLE_VISUALISATIONS =   5#@param {type:"integer"}
#@markdown Save video of population sample over time.
POPULATION_VIDEO = True  #@param (type:"boolean")

USE_EVOLUTION = POP_SIZE > 1

In [ ]:
# @title Saving images on Drive
#@markdown Displayed results can also be stored on Google Drive.
STORE_ON_GOOGLE_DRIVE = False  #@param {type:"boolean"}
GOOGLE_DRIVE_RESULTS_DIR = ""  #@param {type:"string"}

MOUNT_DIR = "/content/drive"

if STORE_ON_GOOGLE_DRIVE:
  from google.colab import drive
  drive.mount(MOUNT_DIR)
  DIR_RESULTS = pathlib.PurePath(MOUNT_DIR, "MyDrive", GOOGLE_DRIVE_RESULTS_DIR)
  print(f"Storing results on Google Drive in {DIR_RESULTS}")
else:
  DIR_RESULTS = "/content"
  print(f"Storing results in Colab in {DIR_RESULTS}")

pathlib.Path(DIR_RESULTS).mkdir(parents=True, exist_ok=True)

# Images patches

In [ ]:
#@title Functions used for loading patches

def add_binary_alpha_mask(patch):
  """Black pixels treated as having alpha=0, all other pixels have alpha=255"""
  shape = patch.shape
  mask = ((patch.sum(2) > 0) * 255).astype(np.uint8)
  return np.concatenate([patch, np.expand_dims(mask, -1)], axis=-1)


def resize_patch(patch, coeff):
  return resize(patch.astype(float),
                (int(np.round(patch.shape[0] * coeff)),
                 int(np.round(patch.shape[1] * coeff))))


def print_size_segmented_data(segmented_data):
  size_max = 0
  shape_max = None
  size_min = np.infty
  shape_min = None
  ws = []
  hs = []
  for i, segment in enumerate(segmented_data):
    segment = segment.swapaxes(0, 1) 
    shape_i = segment.shape
    size_i = shape_i[0] * shape_i[1]
    if size_i > size_max:
      shape_max = shape_i
      size_max = size_i
    if size_i < size_min:
      shape_min = shape_i
      size_min = size_i
    im_i = cv2.cvtColor(segment, cv2.COLOR_RGBA2BGRA)
    im_bgr = im_i[:, :, :3]
    im_mask = np.tile(im_i[:, :, 3:], (1, 1, 3))
    im_render = np.concatenate([im_bgr, im_mask], 1)
    print(f'Patch {i} of shape {shape_i}')
    cv2_imshow(im_render)
  print(f"{len(segmented_data)} patches, max {shape_max}, min {shape_min}")


In [ ]:
#@title Load segmented patches

#@markdown Patch files - select example sets or your own:
PATCH_SET = "Animals" #@param ["Fruit and veg", "Sea glass", "Animals", "Handwritten MNIST", "Upload to Colab", "Load from URL", "Load from Google Drive"]
#@markdown URL if downloading .npy file from website:
URL_TO_PATCH_FILE = "" #@param {type:"string"}
#@markdown Path if loading .npy file from Google Drive:
DRIVE_PATH_TO_PATCH_FILE = "" #@param {type:"string"}

def cached_url_download(url):
  cache_filename = os.path.basename(url)
  cache = pathlib.Path(cache_filename)
  if not cache.is_file():
    print("Downloading " + cache_filename)
    r = requests.get(url)
    bytesio_object = io.BytesIO(r.content)
    with open(cache_filename, "wb") as f:
        f.write(bytesio_object.getbuffer())
  else:
    print("Using cached version of " + cache_filename)
  return np.load(cache, allow_pickle=True)

def upload_file():
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    with open(fn, 'rb') as f:
      return np.load(f, allow_pickle = True)


examples = {"Fruit and veg" : "fruit.npy", 
            "Sea glass" : "shore_glass.npy",
            #"Chrisantha" : "chrisantha2.npy",
            "Handwritten MNIST" : "handwritten_mnist.npy",
            "Animals" : "animals.npy"}

if PATCH_SET in examples:
  repo_root = "https://github.com/deepmind/arnheim/raw/main"
  segmented_data_initial = cached_url_download(
      f"{repo_root}/collage_patches/{examples[PATCH_SET]}")
elif PATCH_SET == "Load from URL":
  segmented_data_initial = cached_url_download(URL_TO_PATCH_FILE)
elif PATCH_SET == "Upload to Colab":
  segmented_data_initial = upload_file()
else:  # "Load from Google Drive"
  drive.mount(MOUNT_DIR)
  data_file = pathlib.PurePath(MOUNT_DIR, "MyDrive", DRIVE_PATH_TO_PATCH_FILE)
  print("Reading", data_file)
  segmented_data_initial = np.load(data_file, allow_pickle=True)


segmented_data_initial_tmp = []
for i in range(len(segmented_data_initial)):
  if segmented_data_initial[i].shape[2] == 3:
    segmented_data_initial_tmp.append(add_binary_alpha_mask(
        segmented_data_initial[i]))
  else:
    segmented_data_initial_tmp.append(
        segmented_data_initial[i])
    
segmented_data_initial = segmented_data_initial_tmp

In [ ]:
#@title Resize image patches to low- and high-res.

NORMALIZE_PATCH_BRIGHTNESS = False  #@param {type: "boolean"}
#@markdown Scale all patches by same amount?
FIXED_SCALE_PATCHES = False #@param {type:"boolean"}
FIXED_SCALE_COEFF =   0.7#@param {type:"number"}

def normalise_patch_brightness(patch):
  max_intensity = max(patch.max(), 1.0)
  return ((patch / max_intensity) * 255).astype(np.uint8)

# Permute the order of the segmented images. 
num_patches = len(segmented_data_initial)
order = np.random.permutation(num_patches)

# Compress all images until they are at most 50% of the large canvas size. 
height_large_max = CANVAS_HEIGHT * MULTIPLIER_BIG_IMAGE / PATCH_MAX_PROPORTION
width_large_max = CANVAS_WIDTH * MULTIPLIER_BIG_IMAGE / PATCH_MAX_PROPORTION
print(
    f"Max patch size on large image: ({height_large_max}, {width_large_max})")
segmented_data = []
segmented_data_high_res = []
for patch_i in range(num_patches):
  segmented_data_initial_i = segmented_data_initial[
      order[patch_i]].astype(np.float32).swapaxes(0, 1)
  shape_i = segmented_data_initial_i.shape
  h_i = shape_i[0]
  w_i = shape_i[1]
  if h_i >= PATCH_HEIGHT_MIN and w_i >= PATCH_WIDTH_MIN:
    # Coefficient for resizing the patch.
    if FIXED_SCALE_PATCHES:
      coeff_i_large = FIXED_SCALE_COEFF
    else:
      coeff_i_large = 1.0
      if h_i > height_large_max:
        coeff_i_large = height_large_max / h_i
      if w_i > width_large_max:
        coeff_i_large = min(coeff_i_large, width_large_max / w_i)

    # Resize the high-res patch?
    if coeff_i_large < 1.0:
      segmented_data_high_res_i = resize_patch(segmented_data_initial_i,
                                              coeff_i_large)
    else:
      segmented_data_high_res_i = np.copy(segmented_data_initial_i)

    # Resize the low-res patch.
    coeff_i = coeff_i_large / MULTIPLIER_BIG_IMAGE
    segmented_data_i = resize_patch(segmented_data_initial_i, coeff_i)
    if NORMALIZE_PATCH_BRIGHTNESS:
      segmented_data_i[...,:3] = normalise_patch_brightness(
          segmented_data_i[...,:3])
      segmented_data_high_res_i[...,:3] = normalise_patch_brightness(
          segmented_data_high_res_i[...,:3])
    segmented_data_high_res_i = segmented_data_high_res_i.astype(np.uint8)
    segmented_data_high_res.append(segmented_data_high_res_i)
    segmented_data_i = segmented_data_i.astype(np.uint8)
    segmented_data.append(segmented_data_i)
    print("{}/{}: initial {} -> small {}, large {} x{:.2f}".format(
        patch_i, num_patches, shape_i, segmented_data_i.shape, 
        segmented_data_high_res_i.shape,
        coeff_i_large))
  else:
    print(f"Discard patch of size {h_i}x{w_i}")

print("Patch sizes during optimisation:")
print_size_segmented_data(segmented_data)
print("Patch sizes for high-resolution final image:")
print_size_segmented_data(segmented_data_high_res)

# Colour and affine transforms

## Affine transform classes

In [ ]:
class PopulationAffineTransforms(torch.nn.Module):
  """Population-based Affine Transform network."""
  def __init__(self, num_patches=1, pop_size=1):
    super(PopulationAffineTransforms, self).__init__()

    self._pop_size = pop_size
    matrices_translation = (
        np.random.rand(pop_size, num_patches, 2, 1) * (MAX_TRANS - MIN_TRANS) 
        + MIN_TRANS)
    matrices_rotation = (
        np.random.rand(pop_size, num_patches, 1, 1) * (MAX_ROT - MIN_ROT)
        + MIN_ROT)
    matrices_scale = (
        np.random.rand(pop_size, num_patches, 1, 1) * (MAX_SCALE - MIN_SCALE) 
        + MIN_SCALE)
    matrices_squeeze = (
        np.random.rand(pop_size, num_patches, 1, 1) * (
            (MAX_SQUEEZE - MIN_SQUEEZE) + MIN_SQUEEZE))
    matrices_shear = (
        np.random.rand(pop_size, num_patches, 1, 1) * (MAX_SHEAR - MIN_SHEAR) 
        + MIN_SHEAR)
    self.translation = torch.nn.Parameter(
        torch.tensor(matrices_translation, dtype=torch.float),
        requires_grad=True)
    self.rotation = torch.nn.Parameter(
        torch.tensor(matrices_rotation, dtype=torch.float),
        requires_grad=True)
    self.scale = torch.nn.Parameter(
        torch.tensor(matrices_scale, dtype=torch.float),
        requires_grad=True)
    self.squeeze = torch.nn.Parameter(
        torch.tensor(matrices_squeeze, dtype=torch.float),
        requires_grad=True)
    self.shear = torch.nn.Parameter(
        torch.tensor(matrices_shear, dtype=torch.float),
        requires_grad=True)
    self._identity = (
        torch.ones((pop_size, num_patches, 1, 1)) * torch.eye(2).unsqueeze(0)
        ).to(device)
    self._zero_column = torch.zeros((pop_size, num_patches, 2, 1)).to(device)
    self._unit_row = (
        torch.ones((pop_size, num_patches, 1, 1)) * torch.tensor([0., 0., 1.])
        ).to(device)
    self._zeros = torch.zeros((pop_size, num_patches, 1, 1)).to(device)

  def _clamp(self):
    self.translation.data = self.translation.data.clamp(
        min=MIN_TRANS, max=MAX_TRANS)
    self.rotation.data = self.rotation.data.clamp(
        min=MIN_ROT, max=MAX_ROT)
    self.scale.data = self.scale.data.clamp(
        min=MIN_SCALE, max=MAX_SCALE)
    self.squeeze.data = self.squeeze.data.clamp(
        min=MIN_SQUEEZE, max=MAX_SQUEEZE)
    self.shear.data = self.shear.data.clamp(
        min=MIN_SHEAR, max=MAX_SHEAR)

  def copy_and_mutate_s(self, parent, child):
    """Copy parameters to child, mutating transform parameters."""
    with torch.no_grad():
      self.translation[child, ...] = (self.translation[parent, ...] 
          + POS_AND_ROT_MUTATION_SCALE * torch.randn(
              self.translation[child, ...].shape).to(device))
      self.rotation[child, ...] = (self.rotation[parent, ...]  
          + POS_AND_ROT_MUTATION_SCALE * torch.randn(
              self.rotation[child, ...].shape).to(device))
      self.scale[child, ...] = (self.scale[parent, ...] 
          + SCALE_MUTATION_SCALE * torch.randn(
              self.scale[child, ...].shape).to(device))
      self.squeeze[child, ...] = (self.squeeze[parent, ...]
          + DISTORT_MUTATION_SCALE * torch.randn(
              self.squeeze[child, ...].shape).to(device))
      self.shear[child, ...] = (self.shear[parent, ...]
          + DISTORT_MUTATION_SCALE * torch.randn(
              self.shear[child, ...].shape).to(device))

  def copy_from(self, other, idx_to, idx_from):
    """Copy parameters from other spatial transform, for selected indices."""
    assert idx_to < self._pop_size
    with torch.no_grad():
      self.translation[idx_to, ...] = other.translation[idx_from, ...]
      self.rotation[idx_to, ...] = other.rotation[idx_from, ...]
      self.scale[idx_to, ...] = other.scale[idx_from, ...]
      self.squeeze[idx_to, ...] = other.squeeze[idx_from, ...]
      self.shear[idx_to, ...] = other.shear[idx_from, ...]

  def forward(self, x):
    self._clamp()
    scale_affine_mat = torch.cat([
        torch.cat([self.scale, self.shear], 3),
        torch.cat([self._zeros, self.scale * self.squeeze], 3)],
        2)
    scale_affine_mat = torch.cat([
        torch.cat([scale_affine_mat, self._zero_column], 3),
        self._unit_row], 2)
    rotation_affine_mat = torch.cat([
        torch.cat([torch.cos(self.rotation), -torch.sin(self.rotation)], 3),
        torch.cat([torch.sin(self.rotation), torch.cos(self.rotation)], 3)],
        2)
    rotation_affine_mat = torch.cat([
        torch.cat([rotation_affine_mat, self._zero_column], 3),
        self._unit_row], 2)
    
    scale_rotation_mat = torch.matmul(scale_affine_mat,
                                      rotation_affine_mat)[:, :, :2, :]
    # Population and patch dimensions (0 and 1) need to be merged.
    # E.g. from (POP_SIZE, NUM_PATCHES, CHANNELS, WIDTH, HEIGHT) 
    # to (POP_SIZE * NUM_PATCHES, CHANNELS, WIDTH, HEIGHT)
    scale_rotation_mat = scale_rotation_mat[:, :, :2, :].view(
        1, -1, *(scale_rotation_mat[:, :, :2, :].size()[2:])).squeeze()
    x = x.view(1, -1, *(x.size()[2:])).squeeze()
    scaled_rotated_grid = F.affine_grid(
        scale_rotation_mat, x.size(), align_corners=True)
    scaled_rotated_x = F.grid_sample(x, scaled_rotated_grid, align_corners=True)

    translation_affine_mat = torch.cat([self._identity, self.translation], 3)
    translation_affine_mat = translation_affine_mat.view(
        1, -1, *(translation_affine_mat.size()[2:])).squeeze()
    translated_grid = F.affine_grid(
        translation_affine_mat, x.size(), align_corners=True)
    y = F.grid_sample(scaled_rotated_x, translated_grid, align_corners=True)
    return y.view(self._pop_size, NUM_PATCHES, *(y.size()[1:]))

  def tensor_to(self, device):
    self.translation = self.translation.to(device)
    self.rotation = self.rotation.to(device)
    self.scale = self.scale.to(device)
    self.squeeze = self.squeeze.to(device)
    self.shear = self.shear.to(device)
    self._identity = self._identity.to(device)
    self._zero_column = self._zero_column.to(device)
    self._unit_row = self._unit_row.to(device)
    self._zeros = self._zeros.to(device)

## RGB and HSV color transforms

In [ ]:
class PopulationOrderOnlyTransforms(torch.nn.Module):

  def __init__(self, num_patches=1, pop_size=1):
    super(PopulationOrderOnlyTransforms, self).__init__()

    self._pop_size = pop_size

    population_zeros = np.ones((pop_size, num_patches, 1, 1, 1))
    population_orders = np.random.rand(pop_size, num_patches, 1, 1, 1)

    self._zeros = torch.nn.Parameter(
        torch.tensor(population_zeros, dtype=torch.float),
        requires_grad=False)
    self.orders = torch.nn.Parameter(
        torch.tensor(population_orders, dtype=torch.float),
        requires_grad=True)
    self._hsv_to_rgb = hsv.HsvToRgb()

  def _clamp(self):
    self.orders.data = self.orders.data.clamp(min=0.0, max=1.0)

  def copy_and_mutate_s(self, parent, child):
    with torch.no_grad():
      self.orders[child, ...] = self.orders[parent, ...]

  def copy_from(self, other, idx_to, idx_from):
    """Copy parameters from other colour transform, for selected indices."""
    assert idx_to < self._pop_size
    with torch.no_grad():
      self.orders[idx_to, ...] = other.orders[idx_from, ...]

  def forward(self, x):
    self._clamp()
    colours = torch.cat(
        [self._zeros, self._zeros, self._zeros, self._zeros, self.orders],
        2)
    return colours * x

  def tensor_to(self, device):
    self.orders = self.orders.to(device)
    self._zeros = self._zeros.to(device)


class PopulationColourHSVTransforms(torch.nn.Module):

  def __init__(self, num_patches=1, pop_size=1):
    super(PopulationColourHSVTransforms, self).__init__()

    print('PopulationColourHSVTransforms for {} patches, {} individuals'.format(
        num_patches, pop_size))
    self._pop_size = pop_size

    coeff_hue = 0.5 * (MAX_HUE - MIN_HUE) + MIN_HUE
    coeff_sat = 0.5 * (MAX_SAT - MIN_SAT) + MIN_SAT
    coeff_val = 0.5 * (MAX_VAL - MIN_VAL) + MIN_VAL
    population_hues = np.random.rand(pop_size, num_patches, 1, 1, 1) * coeff_hue
    population_saturations = np.random.rand(
        pop_size, num_patches, 1, 1, 1) * coeff_sat
    population_values = np.random.rand(
        pop_size, num_patches, 1, 1, 1) * coeff_val
    population_zeros = np.ones((pop_size, num_patches, 1, 1, 1))
    population_orders = np.random.rand(pop_size, num_patches, 1, 1, 1)
    
    self.hues = torch.nn.Parameter(
        torch.tensor(population_hues, dtype=torch.float),
        requires_grad=True)
    self.saturations = torch.nn.Parameter(
        torch.tensor(population_saturations, dtype=torch.float),
        requires_grad=True)
    self.values = torch.nn.Parameter(
        torch.tensor(population_values, dtype=torch.float),
        requires_grad=True)
    self._zeros = torch.nn.Parameter(
        torch.tensor(population_zeros, dtype=torch.float),
        requires_grad=False)
    self.orders = torch.nn.Parameter(
        torch.tensor(population_orders, dtype=torch.float),
        requires_grad=True)
    self._hsv_to_rgb = hsv.HsvToRgb()

  def _clamp(self):
    self.hues.data = self.hues.data.clamp(min=MIN_HUE, max=MAX_HUE)
    self.saturations.data = self.saturations.data.clamp(
        min=MIN_SAT, max=MAX_SAT)
    self.values.data = self.values.data.clamp(min=MIN_VAL, max=MAX_VAL)
    self.orders.data = self.orders.data.clamp(min=0.0, max=1.0)

  def copy_and_mutate_s(self, parent, child):
    with torch.no_grad():
      self.hues[child, ...] = (
          self.hues[parent, ...]
          + COLOUR_MUTATION_SCALE * torch.randn(
              self.hues[child, ...].shape).to(device))
      self.saturations[child, ...] = (
          self.saturations[parent, ...]
          + COLOUR_MUTATION_SCALE * torch.randn(
              self.saturations[child, ...].shape).to(device))
      self.values[child, ...] = (
          self.values[parent, ...]
          + COLOUR_MUTATION_SCALE * torch.randn(
              self.values[child, ...].shape).to(device))
      self.orders[child, ...] = self.orders[parent, ...]

  def copy_from(self, other, idx_to, idx_from):
    """Copy parameters from other colour transform, for selected indices."""
    assert idx_to < self._pop_size
    with torch.no_grad():
      self.hues[idx_to, ...] = other.hues[idx_from, ...]
      self.saturations[idx_to, ...] = other.saturations[idx_from, ...]
      self.values[idx_to, ...] = other.values[idx_from, ...]
      self.orders[idx_to, ...] = other.orders[idx_from, ...]

  def forward(self, image):
    self._clamp()
    colours = torch.cat(
        [self.hues, self.saturations, self.values, self._zeros, self.orders], 2)
    hsv_image = colours * image
    rgb_image = self._hsv_to_rgb(hsv_image[:, :, :3, :, :])
    return torch.cat([rgb_image, hsv_image[:, :, 3:, :, :]], axis=2)

  def tensor_to(self, device):
    self.hues = self.hues.to(device)
    self.saturations = self.saturations.to(device)
    self.values = self.values.to(device)
    self.orders = self.orders.to(device)
    self._zeros = self._zeros.to(device)


class PopulationColourRGBTransforms(torch.nn.Module):

  def __init__(self, num_patches=1, pop_size=1):
    super(PopulationColourRGBTransforms, self).__init__()

    print('PopulationColourRGBTransforms for {} patches, {} individuals'.format(
        num_patches, pop_size))
    self._pop_size = pop_size

    rgb_init_range = INITIAL_MAX_RGB - INITIAL_MIN_RGB
    population_reds = (np.random.rand(pop_size, num_patches, 1, 1, 1) 
        * rgb_init_range) + INITIAL_MIN_RGB
    population_greens = (np.random.rand(
        pop_size, num_patches, 1, 1, 1) * rgb_init_range) + INITIAL_MIN_RGB
    population_blues = (np.random.rand(
        pop_size, num_patches, 1, 1, 1) * rgb_init_range) + INITIAL_MIN_RGB
    population_zeros = np.ones((pop_size, num_patches, 1, 1, 1))
    population_orders = np.random.rand(pop_size, num_patches, 1, 1, 1)

    self.reds = torch.nn.Parameter(
        torch.tensor(population_reds, dtype=torch.float),
        requires_grad=True)
    self.greens = torch.nn.Parameter(
        torch.tensor(population_greens, dtype=torch.float),
        requires_grad=True)
    self.blues = torch.nn.Parameter(
        torch.tensor(population_blues, dtype=torch.float),
        requires_grad=True)
    self._zeros = torch.nn.Parameter(
        torch.tensor(population_zeros, dtype=torch.float),
        requires_grad=False)
    self.orders = torch.nn.Parameter(
        torch.tensor(population_orders, dtype=torch.float),
        requires_grad=True)

  def _clamp(self):
    self.reds.data = self.reds.data.clamp(min=MIN_RGB, max=MAX_RGB)
    self.greens.data = self.greens.data.clamp(min=MIN_RGB, max=MAX_RGB)
    self.blues.data = self.blues.data.clamp(min=MIN_RGB, max=MAX_RGB)
    self.orders.data = self.orders.data.clamp(min=0.0, max=1.0)

  def copy_and_mutate_s(self, parent, child):
    with torch.no_grad():
      self.reds[child, ...] = (
          self.reds[parent, ...] 
          + COLOUR_MUTATION_SCALE * torch.randn(
              self.reds[child, ...].shape).to(device))
      self.greens[child, ...] = (
          self.greens[parent, ...] 
          + COLOUR_MUTATION_SCALE * torch.randn(
              self.greens[child, ...].shape).to(device))
      self.blues[child, ...] = (
          self.blues[parent, ...] 
          + COLOUR_MUTATION_SCALE * torch.randn(
              self.blues[child, ...].shape).to(device))
      self.orders[child, ...] = self.orders[parent, ...] 

  def copy_from(self, other, idx_to, idx_from):
    """Copy parameters from other colour transform, for selected indices."""
    assert idx_to < self._pop_size
    with torch.no_grad():
      self.reds[idx_to, ...] = other.reds[idx_from, ...]
      self.greens[idx_to, ...] = other.greens[idx_from, ...]
      self.blues[idx_to, ...] = other.blues[idx_from, ...]
      self.orders[idx_to, ...] = other.orders[idx_from, ...]

  def forward(self, x):
    self._clamp()
    colours = torch.cat(
        [self.reds, self.greens, self.blues, self._zeros, self.orders], 2)
    return colours * x

  def tensor_to(self, device):
    self.reds = self.reds.to(device)
    self.greens = self.greens.to(device)
    self.blues = self.blues.to(device)
    self.orders = self.orders.to(device)
    self._zeros = self._zeros.to(device)


# Rendering functions

In [ ]:
RENDER_EPSILON = 1e-8
RENDER_OVERLAP_TEMPERATURE = 0.1
RENDER_OVERLAP_ZERO_OFFSET = -5
RENDER_OVERLAP_MASK_THRESHOLD = 0.5
RENDER_TRANSPARENCY_MASK_THRESHOLD = 0.1


def population_render_transparency(x, b=None):
  """Image rendering function that renders all patches on top of one another,
     with transparency, using black as the transparent colour.

  Args:
    x: tensor of transformed RGB image patches of shape [S, B, 5, H, W].
    b: optional tensor of background RGB image of shape [S, 3, H, W].
  Returns:
    Tensor of rendered RGB images of shape [S, 3, H, W].
  """
  # Sum the RGB patches [S, B, 3, H, W] as [S, 3, H, W].
  y = x[:, :, :3, :, :].sum(1)
  if INVERT_COLOURS:
    y[:, :3, :, :] = 1.0 - y[:, :3, :, :]
  # Add backgrounds [S, 3, H, W].
  if b is not None:
    b = b.cuda() if x.is_cuda else b.cpu()
    y = torch.where(y.sum(1, keepdim=True) > RENDER_TRANSPARENCY_MASK_THRESHOLD,
                    y[:, :3, :, :], b.unsqueeze(0)[:, :3, :, :])
  return y.clamp(0., 1.).permute(0, 2, 3, 1)


def population_render_masked_transparency(x, b=None):
  """Image rendering function that renders all patches on top of one another,
     with transparency, using the alpha chanel as the mask colour.

  Args:
    x: tensor of transformed RGB image patches of shape [S, B, 5, H, W].
    b: optional tensor of background RGB image of shape [S, 3, H, W].
  Returns:
    Tensor of rendered RGB images of shape [S, 3, H, W].
  """
  # Get the patch mask [S, B, 1, H, W] and sum of masks [S, 1, H, W].
  mask = x[:, :, 3:4, :, :]
  mask_sum = mask.sum(1) + RENDER_EPSILON
  # Mask the RGB patches [S, B, 4, H, W] -> [S, B, 3, H, W].
  masked_x = x[:, :, :3, :, :] * mask
  # Compute mean of the RGB patches [S, B, 3, H, W] as [S, 3, H, W].
  x_sum = masked_x.sum(1)
  y = torch.where(
      mask_sum > RENDER_EPSILON, x_sum / mask_sum, mask_sum)
  # Anti-aliasing on the countours of the sum of patches.
  y = y * mask_sum.clamp(0., 1.)
  if INVERT_COLOURS:
    y[:, :3, :, :] = 1.0 - y[:, :3, :, :]
  # Add backgrounds [S, 3, H, W].
  if b is not None:
    b = b.cuda() if x.is_cuda else b.cpu()
    y = torch.where(mask.sum(1) > RENDER_OVERLAP_MASK_THRESHOLD, y[:, :3, :, :],
                  b.unsqueeze(0)[:, :3, :, :])
  return y.clamp(0., 1.).permute(0, 2, 3, 1)


def population_render_overlap(x, b=None, gamma=None):
  """Image rendering function that overlays all patches on top of one another,
     with semi-translucent overlap, using the alpha chanel as the mask colour
     and the 5th channel as the order for the overlapped images.

  Args:
    x: tensor of transformed RGB image patches of shape [S, B, 5, H, W].
    b: optional tensor of background RGB image of shape [S, 3, H, W].
  Returns:
    Tensor of rendered RGB images of shape [S, 3, H, W].
  """
  # Get the patch mask [S, B, 1, H, W].
  mask = x[:, :, 3:4, :, :]
  # Mask the patches [S, B, 4, H, W] -> [S, B, 3, H, W]
  masked_x = x[:, :, :3, :, :] * mask * mask
  # Mask the orders [S, B, 1, H, W] -> [S, B, 1, H, W]
  order = torch.where(
      mask > RENDER_OVERLAP_MASK_THRESHOLD,
      x[:, :, 4:, :, :] * mask / RENDER_OVERLAP_TEMPERATURE,
      mask + RENDER_OVERLAP_ZERO_OFFSET)
  # Get weights from orders [S, B, 1, H, W]
  weights = F.softmax(order, dim=1)
  # Apply weights to masked patches and compute mean over patches [S, 3, H, W].
  y = (weights * masked_x).sum(1)
  if INVERT_COLOURS:
    y[:, :3, :, :] = 1.0 - y[:, :3, :, :]
  if b is not None:
    b = b.cuda() if x.is_cuda else b.cpu()
    y = torch.where(mask.sum(1) > RENDER_OVERLAP_MASK_THRESHOLD, y[:, :3, :, :],
                  b.unsqueeze(0)[:, :3, :, :])
  return y.clamp(0., 1.).permute(0, 2, 3, 1)
  

# Collage network definition

In [ ]:
class PopulationCollage(torch.nn.Module):
  """Population-based segmentation collage network."""
  def __init__(self,
               pop_size=1,
               is_high_res=False,
               segmented_data=None,
               background_image=None):
    """Constructor, relying on global parameters."""
    super(PopulationCollage, self).__init__()

    # Population size.
    self._pop_size = pop_size

    # Create the spatial transformer and colour transformer for patches.
    self.spatial_transformer = PopulationAffineTransforms(
        num_patches=NUM_PATCHES, pop_size=pop_size).cuda()
    if COLOUR_TRANSFORMATIONS == "HSV space":
      self.colour_transformer = PopulationColourHSVTransforms(
          num_patches=NUM_PATCHES, pop_size=pop_size).cuda()
    elif COLOUR_TRANSFORMATIONS == "RGB space":
      self.colour_transformer = PopulationColourRGBTransforms(
          num_patches=NUM_PATCHES, pop_size=pop_size).cuda()
    else:
      self.colour_transformer = PopulationOrderOnlyTransforms(
          num_patches=NUM_PATCHES, pop_size=pop_size).cuda()

    # Optimisation is run in low-res, final rendering is in high-res.
    self._high_res = is_high_res

    # Store the background image (low- and high-res).
    self._background_image = background_image
    if self._background_image is not None:
      print(f'Background image of size {self._background_image.shape}')

    # Store the dataset (low- and high-res).
    self._dataset = segmented_data
    #print(f'There are {len(self._dataset)} image patches in the dataset')

    # Initial set of indices, pointing to the NUM_PATCHES first dataset images. 
    self.patch_indices = [np.arange(NUM_PATCHES) % len(self._dataset)
                          for _ in range(pop_size)]

    # Patches in low and high-res.
    self.patches = None
    self.store_patches()

  def store_patches(self, population_idx=None):
    """Store the image patches for each population element."""
    t0 = time.time()

    if population_idx is not None and self.patches is not None:
      list_indices = [population_idx]
      #print(f'Reload {NUM_PATCHES} image patches for [{population_idx}]')
      self.patches[population_idx, :, :4, :, :] = 0
    else:
      list_indices = np.arange(self._pop_size)
      #print(f'Store {NUM_PATCHES} image patches for [1, ..., {self._pop_size}]')
      if self._high_res:
        self.patches = torch.zeros(
            1, NUM_PATCHES, 5, CANVAS_HEIGHT * MULTIPLIER_BIG_IMAGE,
            CANVAS_WIDTH * MULTIPLIER_BIG_IMAGE).to('cpu')
      else:
        self.patches = torch.zeros(
            self._pop_size, NUM_PATCHES, 5, CANVAS_HEIGHT, CANVAS_WIDTH
            ).to(device)
      self.patches[:, :, 4, :, :] = 1.0

    # Put the segmented data into the patches.
    for i in list_indices:
      for j in range(NUM_PATCHES):
        k = self.patch_indices[i][j]
        patch_j = torch.tensor(
            self._dataset[k].swapaxes(0, 2) / 255.0).to(device)
        width_j = patch_j.shape[1]
        height_j = patch_j.shape[2]
        if self._high_res:
          w0 = int((CANVAS_WIDTH * MULTIPLIER_BIG_IMAGE - width_j) / 2.0)
          h0 = int((CANVAS_HEIGHT * MULTIPLIER_BIG_IMAGE - height_j) / 2.0)
        else:
          w0 = int((CANVAS_WIDTH - width_j) / 2.0)
          h0 = int((CANVAS_HEIGHT - height_j) / 2.0)
        self.patches[i, j, :4, w0:(w0 + width_j), h0:(h0 + height_j)] = patch_j
    t1 = time.time()
    #print('Updated patches in {:.3f}s'.format(t1-t0))

  def copy_and_mutate_s(self, parent, child):
    with torch.no_grad():
      # Copy the patches indices from the parent to the child.
      self.patch_indices[child] = copy.deepcopy(self.patch_indices[parent])
      
      # Mutate the child patches with a single swap from the original dataset. 
      if PATCH_MUTATION_PROBABILITY > np.random.uniform():
        idx_dataset  = np.random.randint(len(self._dataset))
        idx_patch  = np.random.randint(NUM_PATCHES)
        self.patch_indices[child][idx_patch] = idx_dataset

      # Update all the patches for the child.
      self.store_patches(child)
  
      self.spatial_transformer.copy_and_mutate_s(parent, child)
      self.colour_transformer.copy_and_mutate_s(parent, child)

  def copy_from(self, other, idx_to, idx_from):
    """Copy parameters from other collage generator, for selected indices."""
    assert idx_to < self._pop_size
    with torch.no_grad():
      self.patch_indices[idx_to] = copy.deepcopy(other.patch_indices[idx_from])
      self.store_patches(idx_to)
      self.spatial_transformer.copy_from(
          other.spatial_transformer, idx_to, idx_from)
      self.colour_transformer.copy_from(
          other.colour_transformer, idx_to, idx_from)

  def forward(self, params=None):
    """Input-less forward function."""

    shifted_patches = self.spatial_transformer(self.patches)
    background_image = self._background_image
    if USE_BACKGROUND == False:
      background_image = None
    coloured_patches = self.colour_transformer(shifted_patches)
    if RENDER_METHOD == "transparency":
      img = population_render_transparency(coloured_patches, background_image)
    elif RENDER_METHOD == "masked_transparency":
      img = population_render_masked_transparency(
          coloured_patches, background_image)
    elif RENDER_METHOD == "opacity":
      if params is not None and 'gamma' in params:
        gamma = params['gamma']
      else:
        gamma = None
      img = population_render_overlap(
          coloured_patches, background_image)
    else:
      print("Unhandled render method")
    return img

  def tensors_to(self, device):
    self.spatial_transformer.tensor_to(device)
    self.colour_transformer.tensor_to(device)
    self.patches = self.patches.to(device)


# Image and video function definitions

In [ ]:
#@title Image rendering and display

def layout_img_batch(img_batch, max_display=None):
    # img_batch.shape = (7, 224, 224, 3)  S, H, W, C
    img_np = img_batch.transpose(0, 2, 1, 3).clip(0.0, 1.0)  # S, W, H, C
    if max_display:
      img_np = img_np[:max_display, ...]
    sp = img_np.shape
    img_np[:, 0, :, :] = 1.0  # White line separator 
    img_stitch = np.reshape(img_np, (sp[1] * sp[0], sp[2], sp[3]))
    img_r = img_stitch.transpose(1, 0, 2)   # H, W, C
    return img_r

def show_and_save(img_batch, t=None,
                  max_display=1, interpolation="None", stitch=True,
                  img_format="SCHW", show=True):
  """Display image.

  Args:
  
    img: image to display
    t: time step
    max_display: max number of images to display from population
    interpolation: interpolate enlarged images
    stitch: append images side-by-side
    img_format: SHWC or SCHW (the latter used by CLIP)

  Returns:
    stitched image or None
  """

  if isinstance(img_batch, torch.Tensor):
    img_np = img_batch.detach().cpu().numpy()
  else:
    img_np = img_batch

  if not stitch:
    print(f"image (not stitch) min {img_np.min()}, max {img_np.max()}")
    for i in range(min(max_display, img_np.shape[0])):
      img = img_np[i]
      if img_format == "SCHW":  # Convert to SHWC
        img = np.transpose(img, (1, 2, 0))
      img = np.clip(img, 0.0, 1.0)
      cv2_imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB) * 255)
      
      type_image = "image_"
      if img.shape[1] > CANVAS_WIDTH:
        type_image = "highres_" + type_image 
      path_fig = f"{DIR_RESULTS}/{type_image}{PROMPT}_{str(i)}"
      if t is not None:
        path_fig += "_t_" + str(t)
      path_fig += ".png"
      if show:
        cv2_imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB) * 255)
      cv2.imwrite(path_fig, img)
    return None
  else:
    print(f"image (stitch) min {img_np.min()}, max {img_np.max()}")
    img_np = np.clip(img_np, 0.0, 1.0)
    num_images = img_np.shape[0]
    if img_format == "SCHW":  # Convert to SHWC
      img_np = img_np.transpose((0, 2, 3, 1))
    laid_out = layout_img_batch(img_np, max_display)
    if show:
      cv2_imshow(cv2.cvtColor(laid_out, cv2.COLOR_BGR2RGB) * 255)
    return laid_out




In [ ]:
#@title Video creator {vertical-output: true}

class VideoWriter:
  """Create a video from image frames."""

  def __init__(self, filename="_autoplay.mp4", fps=20.0, **kw):
    """Video creator.

    Creates and display a video made from frames. The default
    filename causes the video to be displayed on exit.

    Args:
      filename: name of video file
      fps: frames per second for video
      **kw: args to be passed to FFMPEG_VideoWriter

    Returns:
      VideoWriter instance.
    """

    self.writer = None
    self.params = dict(filename=filename, fps=fps, **kw)

  def add(self, img):
    """Add image to video.

    Add new frame to image file, creating VideoWriter if requried.

    Args:
      img: array-like frame, shape [X, Y, 3] or [X, Y]

    Returns:
      None
    """

    img = np.asarray(img)
    if self.writer is None:
      h, w = img.shape[:2]
      self.writer = FFMPEG_VideoWriter(size=(w, h), **self.params)
    if img.dtype in [np.float32, np.float64]:
      img = np.uint8(img.clip(0, 1)*255)
    if len(img.shape) == 2:
      img = np.repeat(img[..., None], 3, -1)
    self.writer.write_frame(img)

  def close(self):
    if self.writer:
      self.writer.close()

  def __enter__(self):
    return self

  def __exit__(self, *kw):
    self.close()
    if self.params["filename"] == "_autoplay.mp4":
      self.show()

  def show(self, **kw):
    """Display video.

    Args:
      **kw: args to be passed to mvp.ipython_display

    Returns:
      None
    """
    self.close()
    fn = self.params["filename"]
    display(mvp.ipython_display(fn, **kw))

In [ ]:
#@title Metadata export

def export_metadata(metadata_filename):
  metadata = {}
  metadata_canvas = {"CANVAS_WIDTH": CANVAS_WIDTH,
                    "CANVAS_HEIGHT": CANVAS_HEIGHT,
                    "MULTIPLIER_BIG_IMAGE": MULTIPLIER_BIG_IMAGE,
                    "PATCH_WIDTH_MIN": PATCH_WIDTH_MIN,
                    "PATCH_HEIGHT_MIN": PATCH_HEIGHT_MIN,
                    "PATCH_MAX_PROPORTION": PATCH_MAX_PROPORTION}
  metadata = {"canvas": metadata_canvas}
  metadata_collage = {"RENDER_METHOD": RENDER_METHOD,
                      "NUM_PATCHES": NUM_PATCHES,
                      "COLOUR_TRANSFORMATIONS": COLOUR_TRANSFORMATIONS,
                      "INVERT_COLOURS": INVERT_COLOURS}
  metadata["collage"] = metadata_collage
  metadata_affine = {"MIN_TRANS": MIN_TRANS,
                    "MAX_TRANS": MAX_TRANS,
                    "MIN_SCALE": MIN_SCALE,
                    "MAX_SCALE": MAX_SCALE,
                    "MIN_SQUEEZE": MIN_SQUEEZE,
                    "MAX_SQUEEZE": MAX_SQUEEZE,
                    "MIN_SHEAR": MIN_SHEAR,
                    "MAX_SHEAR": MAX_SHEAR,
                    "MIN_ROT_DEG": MIN_ROT_DEG,
                    "MAX_ROT_DEG": MAX_ROT_DEG,
                    "MIN_ROT": MIN_ROT,
                    "MAX_ROT": MAX_ROT}
  metadata["affine"] = metadata_affine
  metadata_colour = {"MIN_RGB": MIN_RGB,
                    "MAX_RGB": MAX_RGB,
                    "MIN_HUE": MIN_HUE,
                    "MAX_HUE_DEG": MAX_HUE_DEG,
                    "MAX_HUE": MAX_HUE,
                    "MIN_SAT": MIN_SAT,
                    "MAX_SAT": MAX_SAT,
                    "MIN_VAL": MIN_VAL,
                    "MAX_VAL": MAX_VAL}
  metadata["colour"] = metadata_colour
  metadata_training = {"OPTIM_STEPS": OPTIM_STEPS,
                      "LEARNING_RATE": LEARNING_RATE,
                      "USE_IMAGE_AUGMENTATIONS": USE_IMAGE_AUGMENTATIONS,
                      "NUM_AUGS": NUM_AUGS,
                      "USE_NORMALIZED_CLIP": USE_NORMALIZED_CLIP,
                      "GRADIENT_CLIPPING": GRADIENT_CLIPPING,
                      "INITIAL_SEARCH_SIZE": INITIAL_SEARCH_SIZE}
  metadata["training"] = metadata_training
  metadata_evolution = {"POP_SIZE": POP_SIZE,
                        "EVOLUTION_FREQUENCY": EVOLUTION_FREQUENCY,
                        "GA_METHOD": GA_METHOD,
                        "POS_AND_ROT_MUTATION_SCALE": POS_AND_ROT_MUTATION_SCALE,
                        "SCALE_MUTATION_SCALE": SCALE_MUTATION_SCALE,
                        "DISTORT_MUTATION_SCALE": DISTORT_MUTATION_SCALE,
                        "COLOUR_MUTATION_SCALE": COLOUR_MUTATION_SCALE,
                        "PATCH_MUTATION_PROBABILITY": PATCH_MUTATION_PROBABILITY,
                        "MAX_MULTIPLE_VISUALISATIONS": MAX_MULTIPLE_VISUALISATIONS,
                        "USE_EVOLUTION": USE_EVOLUTION}
  metadata["evolution"] = metadata_evolution
  metadata_drive = {"STORE_ON_GOOGLE_DRIVE": STORE_ON_GOOGLE_DRIVE,
                    "GOOGLE_DRIVE_RESULTS_DIR": GOOGLE_DRIVE_RESULTS_DIR}
  metadata["drive"] = metadata_drive
  metadata_patches = {"PATCH_SET": PATCH_SET,
                      "URL_TO_PATCH_FILE": URL_TO_PATCH_FILE,
                      "DRIVE_PATH_TO_PATCH_FILE": DRIVE_PATH_TO_PATCH_FILE,
                      "NORMALIZE_PATCH_BRIGHTNESS": NORMALIZE_PATCH_BRIGHTNESS,
                      "FIXED_SCALE_PATCHES": FIXED_SCALE_PATCHES,
                      "FIXED_SCALE_COEFF": FIXED_SCALE_COEFF}
  metadata["patches"] = metadata_patches

  metadata_prompt = {"PROMPT": PROMPT,
                    "FILE_BASENAME": FILE_BASENAME,
                    "USE_BACKGROUND": USE_BACKGROUND,
                    "VIDEO_STEPS": VIDEO_STEPS,
                    "TRACE_EVERY": TRACE_EVERY}
  metadata["prompt"] = metadata_prompt

  # Write metadata to a Python-interpretable text file.
  with open(metadata_filename, "w") as f:
    for config_key, config in metadata.items():
      f.write(f"# {config_key}\n")
      for key, value in config.items():
        if isinstance(value, str):
          f.write(f"{key} = \"{value}\"\n")
        else:
          f.write(f"{key} = {value}\n")

# Training and evolution function definitions

In [ ]:
#@title Image augmentation transformations for training

def augmentation_transforms(canvas_width,
                            use_normalized_clip=False,
                            use_augmentation=False):
  """Image transforms to produce distorted crops to augment the evaluation.

  Args:
    canvas_width: width of the drawing canvas
    use_normalized_clip: Normalisation to better suit CLIP's training data
    use_augmentation: Image augmentation by affine transform

  Returns:
    transforms
  """
  if use_normalized_clip and use_augmentation:
    augment_trans = transforms.Compose(
        [transforms.RandomPerspective(fill=1, p=1, distortion_scale=0.6),
         transforms.RandomResizedCrop(canvas_width, scale=(0.7, 0.9)),
         transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                              (0.26862954, 0.26130258, 0.27577711))])
  elif use_augmentation:
    augment_trans = transforms.Compose([
        transforms.RandomPerspective(fill=1, p=1, distortion_scale=0.6),
        transforms.RandomResizedCrop(canvas_width, scale=(0.7, 0.9)),
    ])
  elif use_normalized_clip:
    augment_trans = transforms.Normalize(
        (0.48145466, 0.4578275, 0.40821073),
        (0.26862954, 0.26130258, 0.27577711))
  else:
    augment_trans = transforms.RandomPerspective(
        fill=1, p=0, distortion_scale=0)

  return augment_trans

In [ ]:
#@title Training functions

def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n


def plot_and_save_losses(loss_history, title="Losses", filename=None):
  losses = np.array(loss_history)
  plt.figure(figsize=(10,10))
  plt.xlabel("Training steps")
  plt.ylabel("Loss")
  plt.title(title)
  plt.plot(moving_average(losses, n=3))
  if filename:
    np.save(filename + ".npy", losses, allow_pickle=True)
    plt.savefig(filename + ".png")
  plt.show()


def make_optimizer(generator, learning_rate):
  """Make optimizer for generator's parameters.

  Args:
    generator: generator model
    learning_rate: learning rate
    input_learing_rate: learning rate for input

  Returns:
    optimizer
  """

  my_list = ['positions_top']
  params = list(map(lambda x: x[1],list(filter(lambda kv: kv[0] in my_list, 
                                               generator.named_parameters()))))
  base_params = list(map(lambda x: x[1],list(filter(lambda kv: kv[0] not in 
                                      my_list, generator.named_parameters()))))
  lr_scheduler = torch.optim.SGD([{'params': base_params},
                                  {'params': params, 'lr': learning_rate}],
                                    lr=learning_rate)
  return lr_scheduler


def text_features(prompts):
  # Compute CLIP features for all prompts.
  text_inputs = []
  for prompt in prompts:
    text_inputs.append(clip.tokenize(prompt).to(device))

  features = []
  with torch.no_grad():
    for text_input in text_inputs:
      features.append(clip_model.encode_text(text_input))
  return features


def create_augmented_batch(images, augment_trans, text_features):
  """Create batch of images to be evaluated.
  
  Returns:
    img_batch: For compositional images the batch contains all the regions.
        Otherwise the batch contains augmented versions of the original images.
    num_augs: number of images per original image
    expanded_text_features: a text feature for each augmentation
    loss_weights: weights for loss associated with each augmentation image
  """
  images = images.permute(0, 3, 1, 2)  # NHWC -> NCHW
  expanded_text_features = []
  if USE_IMAGE_AUGMENTATIONS:
    num_augs = NUM_AUGS
    img_augs = [] 
    for n in range(NUM_AUGS):
      img_n = augment_trans(images)
      img_augs.append(img_n)
      expanded_text_features.append(text_features[0])
    img_batch = torch.cat(img_augs)
    # Given images [P0, P1] and augmentations [a0(), a1()], output format:
    # [a0(P0), a0(P1), a1(P0), a1(P1)]
  else:
    num_augs = 1
    if USE_NORMALIZED_CLIP:
      img_batch = augment_trans(images)
    else:
      img_batch = images
    expanded_text_features.append(text_features[0])
  return img_batch, num_augs, expanded_text_features, [1] * NUM_AUGS


def create_compositional_batch(images, augment_trans, text_features):
  """Create 10 sub-images per image by augmenting each with 3x3 crops.

  Args:
    images: population of N images, format [N, C, H, W]

  Returns:
    Tensor of all compositional sub-images + originals; [N*10, C, H, W] format:
        [x0_y0(P0) ... x0_y0(PN), ..., x2_y2(P0) ... x2_y2(PN), P0, ..., PN]
    10: Number of sub-images + whole, per original image.
    expanded_text_features: list of text features, 1 for each composition image
    loss_weights: weights for the losses corresponding to each composition image
    """
  if len(text_features) != 10:
    # text_features should already be 10 in size.
    raise ValueError(
        "10 text prompts required for compositional image creation")
  resize_for_clip = transforms.Compose([transforms.Scale((224,224))])
  img_swap = torch.swapaxes(images, 3, 1)
  ims = []
  i = 0
  for x in range(3):
    for y in range(3):
      #  print(f"Prompt for x={x}, y={y} is {PROMPTS[i]}")
      for k in range(images.shape[0]):
        ims.append(resize_for_clip(
            img_swap[k][:, y * 112 : y * 112 + 224, x * 112 : x * 112 + 224]))  
      i += 1
  
  # Top-level (whole) images
  for k in range(images.shape[0]):
    ims.append(resize_for_clip(img_swap[k]))
  all_img = torch.stack(ims)
  all_img = torch.swapaxes(all_img, 1, 3)
  all_img = all_img.permute(0, 3, 1, 2)  # NHWC -> NCHW
  all_img = augment_trans(all_img)

  # Last image gets 9 times as much weight
  common_weight = 1 / 5
  loss_weights = [common_weight] * 9
  loss_weights.append(9 * common_weight)
  return all_img, 10, text_features, loss_weights

In [ ]:
#@title Evaluation and step of optimization

# Show each image being evaluated for debugging purposes.
VISUALISE_BATCH_IMAGES = False

def evaluation(t, clip_enc, generator, augment_trans, text_features):
  """Do a step of evaluation, returning images and losses.

  Args:
    t: step count
    clip_enc: model for CLIP encoding
    generator: drawing generator to optimise
    augment_trans: transforms for image augmentation
    text_features: tuple with the prompt two negative prompts
  
  Returns:
    loss: torch.Tensor of single combines loss
    losses_separate_np: numpy array of loss for each image
    losses_individuals_np: numpy array with loss for each population individual
    img_np: numpy array of images from the generator
  """

  # Annealing parameters.
  params = {'gamma': t / OPTIM_STEPS}

  # Rebuild the generator.
  img = generator(params)
  img_np = img.detach().cpu().numpy()

  # Create images for different regions
  pop_size = img.shape[0]
  if COMPOSITIONAL_IMAGE:
    (img_batch, num_augs, text_features, loss_weights
     ) = create_compositional_batch(img, augment_trans, text_features)
  else:
    (img_batch, num_augs, text_features, loss_weights
     ) = create_augmented_batch(img, augment_trans, text_features)
  losses = torch.zeros(pop_size, num_augs).to(device)

  # Compute and add losses after augmenting the image with transforms.
  img_batch = torch.clip(img_batch, 0, 1)  # clip the images.
  image_features = clip_enc.encode_image(img_batch)
  count = 0
  for n in range(num_augs):  # number of augmentations or composition images
    for p in range(pop_size):
      loss = torch.cosine_similarity(
          text_features[n], image_features[count:count+1], dim=1
          )[0] * loss_weights[n]
      losses[p, n] -= loss
      if VISUALISE_BATCH_IMAGES and t % 500 == 0:
        # Show all the images in the batch along with their losses.
        if COMPOSITIONAL_IMAGE:
          print(f"Loss {loss} for image region with prompt {PROMPTS[n]}:")
        else:
          print(f"Loss {loss} for image augmentation with prompt {PROMPTS[0]}:")
        show_and_save(img_batch[count].unsqueeze(0), img_format="SCHW")
      count += 1
  loss = torch.sum(losses) / pop_size
  losses_separate_np = losses.detach().cpu().numpy()
  # Sum losses for all each population individual.
  losses_individuals_np = losses_separate_np.sum(axis=1)
  return loss, losses_separate_np, losses_individuals_np, img_np


def step_optimization(t, clip_enc, lr_scheduler, generator, augment_trans,
                      text_features, final_step=False):
  """Do a step of optimization.

  Args:
    t: step count
    clip_enc: model for CLIP encoding
    lr_scheduler: optimizer
    generator: drawing generator to optimise
    augment_trans: transforms for image augmentation
    text_features: list or 1 or 9 prompts for normal and compositional creation
    final_step: if True does extras such as saving the model

  Returns:
    losses_np: numpy array with loss for each population individual
    losses_separate_np: numpy array of loss for each image
  """

  # Anneal learning rate and other parameters.
  if t == int(OPTIM_STEPS / 3):
    for g in lr_scheduler.param_groups:
      g["lr"] = g["lr"] / 2.0
  if t == int(OPTIM_STEPS * (2/3)):
    for g in lr_scheduler.param_groups:
      g["lr"] = g["lr"] / 2.0
  params = {'gamma': t / OPTIM_STEPS}

  # Forward pass. 
  lr_scheduler.zero_grad()
  loss, losses_separate_np, losses_np, img_np = evaluation(
      t, clip_enc, generator, augment_trans, text_features)

  # Backpropagate the gradients.
  loss.backward()
  torch.nn.utils.clip_grad_norm(generator.parameters(), GRADIENT_CLIPPING)
  
  # Decay the learning rate.
  lr_scheduler.step()

  # Render the big version.
  if final_step:
    show_and_save(img_np, t=t, img_format="SHWC")
    print("Saving model...")
    torch.save(generator.state_dict(), DIR_RESULTS + "/generator.pt")

  if t % TRACE_EVERY == 0:
    img_stitched = show_and_save(img_np, 
                                 max_display=MAX_MULTIPLE_VISUALISATIONS,
                                 stitch=True, img_format="SHWC")

    print("Iteration {:3d}, rendering loss {:.6f}".format(t, loss.item()))
  return losses_np, losses_separate_np, img_np

In [ ]:
#@title Evolution functions

def population_evolution_step(generator, losses):
  """GA for the population."""
  if GA_METHOD == "Microbial":
    # Competition between 2 random individuals; mutated winner replaces loser.
    indices = list(range(len(losses)))
    random.shuffle(indices)
    select_1, select_2 = indices[0], indices[1]
    if losses[select_1] < losses[select_2]:
      # print(f"Replacing {select_2} with {select_1}")
      generator.copy_and_mutate_s(select_1, select_2)
    else:
      # print(f"Replacing {select_1} with {select_2}")
      generator.copy_and_mutate_s(select_2, select_1)
  elif GA_METHOD == "Evolutionary Strategies":
    # Replace rest of population with mutants of the best."""
    winner = np.argmin(losses)
    for other in range(len(losses)):
      if other == winner:
        continue
      generator.copy_and_mutate_s(winner, other)

# Make Collages

In [ ]:
#@title CollageMaker class definition.

class CollageMaker():
  def __init__(self, #prompts, 
               segmented_data, 
               background_image,
               output_dir,
               file_basename, 
               video_steps,
               population_video):
    #self._prompts = prompts
    self._segmented_data = segmented_data
    self._background_image = background_image
    self._file_basename = file_basename
    self._output_dir = output_dir
    self._population_video = population_video

    self._video_steps = video_steps
    if self._video_steps:
      self._video_writer = VideoWriter(
          filename=f"{self._output_dir}/{self._file_basename}.mp4")
      if self._population_video:
        self._population_video_writer = VideoWriter(
            filename=f"{self._output_dir}/{self._file_basename}_pop_sample.mp4")
    
    if COMPOSITIONAL_IMAGE:
      print("Global prompt is", PROMPTS[-1])
      print("Composition prompts", PROMPTS)
    else:
      print("CLIP prompt", PROMPTS)
    
    # Prompt to CLIP features.
    self._prompt_features = text_features(PROMPTS)
    self._augmentations = augmentation_transforms(
        224,
        use_normalized_clip=USE_NORMALIZED_CLIP,
        use_augmentation=USE_IMAGE_AUGMENTATIONS)
    
    # Create population of collage generators.
    self._generator = PopulationCollage(
        is_high_res=False,
        pop_size=POP_SIZE,
        segmented_data=segmented_data,
        background_image=background_image)
    
    # Initial search over hyper-parameters.
    if INITIAL_SEARCH_SIZE > 1:
      print(f'\nInitial random search over {INITIAL_SEARCH_SIZE} individuals')
      for j in range(POP_SIZE):
        generator_search = PopulationCollage(
            is_high_res=False,
            pop_size=INITIAL_SEARCH_SIZE,
            segmented_data=segmented_data,
            background_image=background_image)
        _, _, losses, _ = evaluation(
            0, clip_model, generator_search, augmentations, prompt_features)
        print(f"Search {losses}")
        idx_best = np.argmin(losses)
        generator.copy_from(generator_search, j, idx_best)
        del generator_search
      print(f'Initial random search done\n')
    
    self._optimizer = make_optimizer(self._generator, LEARNING_RATE)
    self._step = 0
    self._losses_history = []
    self._losses_separated_history = []

  @property
  def generator(self):
    return self._generator
    
  @property
  def step(self):
    return self._step

  def loop(self):
    """Main optimisation/image generation loop. Can be interrupted."""
    if self._step == 0:
      print('Starting optimization of collage.')
    else:
      print(f'Continuing optimization of collage at step {self._step}.')
      if self._video_steps:
        print(f"Aborting video creation (does not work when interrupted).")
        self._video_steps = 0
        self._video_writer = None
        if self._population_video_writer:
          self._population_video_writer = None
    
    while self._step < OPTIM_STEPS:
      last_step = self._step == (OPTIM_STEPS - 1)
      losses, losses_separated, img_batch = step_optimization(
          self._step, clip_model, self._optimizer, self._generator,
          self._augmentations, self._prompt_features, final_step=last_step)
      self._add_video_frames(img_batch, losses)
      self._losses_history.append(losses)
      self._losses_separated_history.append(losses_separated)
    
      if USE_EVOLUTION and self._step and self._step % EVOLUTION_FREQUENCY == 0:
        population_evolution_step(self._generator, losses) 
      self._step += 1

  def high_res_render(self, 
                      segmented_data_high_res, 
                      background_image_high_res,
                      gamma=1.0, 
                      show=True):
    """Save and/or show a high res render using high-res patches."""
    generator = PopulationCollage(
        is_high_res=True,
        pop_size=1,
        segmented_data=segmented_data_high_res,
        background_image=background_image_high_res)
    idx_best = np.argmin(self._losses_history[-1])
    print(f'Lowest loss for indices: {idx_best}')
    generator.copy_from(self._generator, 0, idx_best)
    # Show high res version given a generator
    generator_cpu = copy.deepcopy(generator)
    generator_cpu = generator_cpu.to('cpu')
    generator_cpu.tensors_to('cpu')
  
    params = {'gamma': gamma}
    with torch.no_grad():
      img_high_res = generator_cpu.forward(params)
    img = img_high_res.detach().cpu().numpy()[0]
  
    # Swap Red with Blue
    img = img[...,[2, 1, 0]]  
    img = np.clip(img, 0.0, 1.0) * 255
    image_filename = f"{self._output_dir}/{self._file_basename}.png"
    cv2.imwrite(image_filename, img)
    
    if show:
      cv2_imshow(img)
      cv2.waitKey()

  def finish(self):
    """Finish video writing and save all other data."""
    if self._losses_history:
      losses_filename = f"{self._output_dir}/{self._file_basename}_losses"
      plot_and_save_losses(self._losses_history, 
                           title=f"{PROMPT} ({RENDER_METHOD}) Losses",
                           filename=losses_filename)
    if self._video_steps:
      self._video_writer.close()
    if self._population_video:
      self._population_video_writer.close()
    metadata_filename = f"{self._output_dir}/{self._file_basename}_metadata.py"
    export_metadata(metadata_filename)

  def _add_video_frames(self, img_batch, losses):
    """Add images from numpy image batch to video writers.
    
    Args:
      img_batch: numpy array, batch of images (S,H,W,C)
      losses: numpy array, losses for each generator (S,N)
    """
    if self._video_steps and self._step % self._video_steps == 0:
      # Write image to video.
      best_img = img_batch[np.argmin(losses)]
      self._video_writer.add(cv2.resize(
          best_img, (best_img.shape[1] * 3, best_img.shape[0] * 3)))
      if self._population_video:
        laid_out = layout_img_batch(img_batch)
        self._population_video_writer.add(cv2.resize(
            laid_out, (laid_out.shape[1] * 2, laid_out.shape[0] * 2)))

In [ ]:
# @title Configure image prompt and content

#@markdown Enter a description of the image, e.g. 'a photorealistic chicken'
# PROMPT = "A photorealistic chicken"  #@param {type:"string"}

PROMPT = "A photorealistic landscape"   #@param {type:"string"}

FILE_BASENAME = "landscape_composition"  #@param {type:"string"}

#@markdown Use an uploaded image or solid colour as background instead of black
USE_BACKGROUND = False #@param {type:"boolean"}

#@markdown Use solid colour instead of an image?
USE_SOLID_COLOUR_BACKGROUND = True #@param {type:"boolean"}
BACKGROUND_RED = 195 #@param {type:"slider", min:0, max:255, step:1}
BACKGROUND_GREEN = 181 #@param {type:"slider", min:0, max:255, step:1}
BACKGROUND_BLUE = 172 #@param {type:"slider", min:0, max:255, step:1}

#@markdown Debugging and monitoring
VIDEO_STEPS = 10  #@param {type:"integer"}
TRACE_EVERY = 100  #@param {type:"integer"}

PROMPTS = [PROMPT]

In [ ]:
# @title Configure region prompts

#@markdown If using 3x3 compositional image specify prompt for each region (left to right, starting at the top)
PROMPT_x0_y0 = "a photorealistic sky with sun"   #@param {type:"string"}
PROMPT_x1_y0 = "a photorealistic sky"   #@param {type:"string"}
PROMPT_x2_y0 = "a photorealistic sky with moon"   #@param {type:"string"}
PROMPT_x0_y1 = "a photorealistic tree"   #@param {type:"string"}
PROMPT_x1_y1 = "a photorealistic tree"   #@param {type:"string"}
PROMPT_x2_y1 = "a photorealistic tree"   #@param {type:"string"}
PROMPT_x0_y2 = "a photorealistic field"   #@param {type:"string"}
PROMPT_x1_y2 = "a photorealistic field"   #@param {type:"string"}
PROMPT_x2_y2 = "a photorealistic chicken"   #@param {type:"string"}

if COMPOSITIONAL_IMAGE:
  PROMPTS = [PROMPT_x0_y0, PROMPT_x1_y0, PROMPT_x2_y0, PROMPT_x0_y1, PROMPT_x1_y1, PROMPT_x2_y1, PROMPT_x0_y2, PROMPT_x1_y2, PROMPT_x2_y2, PROMPT]

In [ ]:
#@title Get background image (if using one)

def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())

def _resize_background(img, canvas_height, canvas_width):
  img = torch.tensor(resize(img.astype(float),
                            (canvas_height, canvas_width))).cuda()
  return img.permute(2, 0, 1).to(torch.float32) / 255.0

background_image = None
background_image_high_res = None

if USE_BACKGROUND:
  if not USE_SOLID_COLOUR_BACKGROUND:
    backgrounds = upload_files()
    img = cv2.imread(backgrounds[0])
    cv2_imshow(img)
    cv2.waitKey()
    
    # Convert to numpy array for use. 
    background_image_np = np.asarray(img)
    background_image_np = background_image_np[..., ::-1].copy()
    print('Loaded background image')
  else:
    background_image_np = np.ones((10, 10, 4), dtype=np.float32)
    background_image_np[:, :, 0] = BACKGROUND_RED
    background_image_np[:, :, 1] = BACKGROUND_GREEN
    background_image_np[:, :, 2] = BACKGROUND_BLUE
    background_image_np[:, :, 3] = 255.
    print('Defined background colour ({}, {}, {})'.format(
        BACKGROUND_RED, BACKGROUND_GREEN, BACKGROUND_BLUE))

  background_image = _resize_background(
      background_image_np, CANVAS_HEIGHT, CANVAS_WIDTH)
  background_image_high_res = _resize_background(
      background_image_np,
      CANVAS_HEIGHT * MULTIPLIER_BIG_IMAGE,
      CANVAS_WIDTH * MULTIPLIER_BIG_IMAGE)


In [ ]:
#@title Create collage! (Initialisation)

OUTPUT_DIR = f"{FILE_BASENAME}_results"
!mkdir {OUTPUT_DIR}
!rm {OUTPUT_DIR}/*

# Prepare the outputs.
zipname = f"{FILE_BASENAME}_rendering.zip"
print(f"Output to {OUTPUT_DIR}, zipfile will be saved to {zipname}")
file_basename = f"{FILE_BASENAME}_{RENDER_METHOD}"

collage_maker = CollageMaker(#prompts, 
               segmented_data=segmented_data, background_image=background_image,
               output_dir=OUTPUT_DIR, file_basename=file_basename, 
               video_steps=VIDEO_STEPS, population_video=POPULATION_VIDEO)


In [ ]:
#@title Create collage! (Main loop)

#@markdown To edit patches interrupt this cell and run the one below this. Re-run this cell afterwards to continue generating the image.
collage_maker.loop()

In [ ]:
#@title Tinker with patches
#@markdown Enable this cell to allow patch editing:
PATCH_TINKERING = False #@param {type:"boolean"}

#@markdown Interupt the cell above mid-optimisation and run this cell to manually adjust the patches. Run it several times to adjust different patches. Then re-run the cell above to continue optimising.

if PATCH_TINKERING:
  from ipywidgets import interactive
  import IPython.display
  from google.colab.output import eval_js
  import base64
  
  # Render the current collage(s).
  generator = collage_maker.generator
  step = collage_maker.step
  params = {'gamma': step / OPTIM_STEPS}
  img = generator(params)
  img = img.permute(0, 3, 1, 2)  # NHWC -> NCHW
  print('Current collage(s)')
  res_img = show_and_save(img, t=step,
                          max_display=MAX_MULTIPLE_VISUALISATIONS,
                          stitch=True, show=False)
  filename_temp = f"./temp.png"
  res_img = cv2.cvtColor(res_img, cv2.COLOR_BGR2RGB) * 255
  cv2.imwrite(filename_temp, res_img)
  
  # HTML code to plot the image and detect the mouse cursor.
  canvas_html = """
  <canvas width=%d height=%d></canvas>
  <script>
    var filename_image = "%s"
    var canvas = document.querySelector('canvas')
    var ctx = canvas.getContext('2d')
    ctx.lineWidth = 1
    var mouse = {x: 0, y: 0}
    canvas.addEventListener('mousemove', function(e) {
      mouse.x = e.pageX - this.offsetLeft
      mouse.y = e.pageY - this.offsetTop
    })
    canvas.onmousedown = ()=>{
      ctx.beginPath()
      ctx.moveTo(mouse.x, mouse.y)
      canvas.addEventListener('mousemove', onPaint)
    }
    var onPaint = ()=>{
      ctx.lineTo(mouse.x, mouse.y)
      ctx.stroke()
    }
    var data = new Promise(resolve=>{
      canvas.onmouseup = ()=>{
        canvas.removeEventListener('mousemove', onPaint)
        resolve(mouse)
      }
    })
    function draw_collage_image() {
      collage_image = new Image();
      collage_image.src = filename_image;
      collage_image.onload = function(){
        ctx.drawImage(collage_image, 0, 0);
      }
    }
    draw_collage_image();
  </script>
  """
  
  im = IPython.display.Image(filename_temp, embed=True)
  # IPython.display.display(im)
  filename_embed = 'data:image/png;base64,'
  filename_embed += base64.b64encode(im.data).decode('ascii')
  
  # Display an HTML canvas with the image.
  canvas = IPython.display.HTML(
      canvas_html % (CANVAS_WIDTH * POP_SIZE, CANVAS_HEIGHT, filename_embed))
  print('Click with the mouse on the desired image and patch:')
  IPython.display.display(canvas)
  
  # Select the image and pixel coordinates.
  def draw():
    print('draw()')
    mouse = eval_js('data')
    return mouse
  mouse = draw()
  pop_id_mouse = int(np.floor(mouse['x'] / CANVAS_WIDTH))
  x_mouse = int(mouse['x'] % CANVAS_WIDTH)
  y_mouse = int(mouse['y'])
  print(f'Selected image {pop_id_mouse} at ({x_mouse}, {y_mouse})')
  
  def find_patch(generator, id, u, v):
    # Render only the spatial transforms of the patches.
    rendered_patches = generator.spatial_transformer(generator.patches)
    rendered_patches = rendered_patches.detach().cpu().numpy()
    patch_id = np.argmax(rendered_patches[id, :, 3, u, v] * rendered_patches[id, :, 4, u, v])
    return patch_id
  
  # Select the patch.
  patch_id = find_patch(generator, pop_id_mouse, y_mouse, x_mouse)
  print(f'Found matching patch {patch_id}')
  
  # Extract the patch's current affine transform paramaters.
  with torch.no_grad():
    x0 = generator.spatial_transformer.translation[pop_id_mouse, patch_id, 0, 0]
    x0 = float(x0.detach().cpu().numpy())
    y0 = generator.spatial_transformer.translation[pop_id_mouse, patch_id, 1, 0]
    y0 = float(y0.detach().cpu().numpy())
    rot0 = generator.spatial_transformer.rotation[pop_id_mouse, patch_id, 0, 0]
    rot0 = float(rot0.detach().cpu().numpy())
    scale0 = generator.spatial_transformer.scale[pop_id_mouse, patch_id, 0, 0]
    scale0 = float(scale0.detach().cpu().numpy())
    squeeze0 = generator.spatial_transformer.squeeze[pop_id_mouse, patch_id, 0, 0]
    squeeze0 = float(squeeze0.detach().cpu().numpy())
    shear0 = generator.spatial_transformer.shear[pop_id_mouse, patch_id, 0, 0]
    shear0 = float(shear0.detach().cpu().numpy())
    patch_info = {'pop_id': pop_id_mouse, 'patch_id': patch_id,
                  'x0': x0, 'y0': y0, 'rot0': rot0,
                  'scale0': scale0, 'squeeze0': squeeze0, 'shear0': shear0,
                  'x': x0, 'y': y0, 'rot': rot0,
                  'scale': scale0, 'squeeze': squeeze0, 'shear': shear0}
  
  def show_modified(dx, dy, drot, dscale, dsqueeze, dshear):
    """Visualization callback function with affine transform deltas."""
    with torch.no_grad():
      x = patch_info['x0'] - dx
      y = patch_info['y0'] + dy
      rot = patch_info['rot0'] - drot
      scale = patch_info['scale0'] - dscale
      squeeze = patch_info['squeeze0'] + dsqueeze
      shear = patch_info['shear0'] + dshear
      generator.spatial_transformer.translation[pop_id_mouse, patch_id, 0, 0] = x
      generator.spatial_transformer.translation[pop_id_mouse, patch_id, 1, 0] = y
      generator.spatial_transformer.rotation[pop_id_mouse, patch_id, 0, 0] = rot
      generator.spatial_transformer.scale[pop_id_mouse, patch_id, 0, 0] = scale
      generator.spatial_transformer.squeeze[pop_id_mouse, patch_id, 0, 0] = squeeze
      generator.spatial_transformer.shear[pop_id_mouse, patch_id, 0, 0] = shear
    patch_info['x'] = x
    patch_info['y'] = y
    patch_info['rot'] = rot
    patch_info['shear'] = shear
    patch_info['squeeze'] = squeeze
    patch_info['shear'] = shear
    params = {'gamma': step / OPTIM_STEPS}
    img = generator(params)
    img = img.permute(0, 3, 1, 2)  # NHWC -> NCHW
    _ = show_and_save(img, t=step,
                      max_display=MAX_MULTIPLE_VISUALISATIONS,
                      stitch=True)
  
  # Interactive editing of the patch's affine transform parameters.
  interactive_plot = interactive(show_modified,
                                dx=(-MAX_TRANS * 2, MAX_TRANS * 2, 0.01),
                                dy=(-MAX_TRANS * 2, MAX_TRANS * 2, 0.01),
                                drot=(-MAX_ROT * 2, MAX_ROT * 2, 0.01),
                                dscale=(-MAX_SCALE * 2, MAX_SCALE * 2, 0.01),
                                dsqueeze=(-MAX_SQUEEZE * 2, MAX_SQUEEZE * 2, 0.01),
                                dshear=(-MAX_SHEAR * 2, MAX_SHEAR * 2, 0.01))
  output = interactive_plot.children[-1]
  output.layout.height = '350px'
else:
  interactive_plot = "Patch tinkering not enabled."
interactive_plot
  

In [ ]:
#@title Render high res image and finish up.

collage_maker.high_res_render(segmented_data_high_res,
                              background_image_high_res)
collage_maker.finish()

# Download all the generated files.
!zip -r {zipname} {OUTPUT_DIR}
from google.colab import files
files.download(zipname)

# Extras

In [ ]:
raise ValueError("Stop here.")

In [ ]:
#@title zip up outputs and download
!zip -r {zipname} {OUTPUT_DIR}
from google.colab import files
files.download(zipname)

In [ ]:
#@title Upload the image patches
#@markdown Run this cell to upload a npy file containing segmented patches.

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  with open(fn, 'rb') as f:
    segmented_data_initial = np.load(f, allow_pickle = True)


In [ ]:
#@title Creates patch file from PNGs
#@markdown Colab needs to be changed to support local npy files.

import imageio
import glob

TARGET_FILE = "/content/patches.npy"
PNG_DIR = "/content/pngs"
mkdir(PNG_DIR)

def upload_files(target_path):
  """Upload files to target directory."""
  mkdir(target_path)
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(target_path + "/" + k, 'wb').write(v)
  return list(uploaded.keys())

png_imgs = []
for png_im_path in glob.glob(PNG_DIR + "/*.png"):
     png_im = imageio.imread(png_im_path)
     print(png_im.shape)
     png_imgs.append(png_im)

png_imgs_np = np.array(png_imgs)
np.save(TARGET_FILE, png_imgs_np, allow_pickle=True)


#Scratch Space

In [ ]:
#@title Render image 0 and 1 with each rendering method

rms = ["opacity", "transparency", "masked_transparency"]
for rm in rms:
  RENDER_METHOD = rm
  print(rm)
  t = 1
  params = {'gamma': t}
  img = generator(params)
  img = img.permute(0, 3, 1, 2)  # NHWC -> NCHW
  show_and_save(img, t=t, max_display=2)


In [ ]:
#@title Configure tiling
# TILES_WIDTH = 1  #@param {type:"slider", min:1, max:8}
# TILES_HEIGHT = 1  #@param {type:"slider", min:1, max:8}
# TILES_PROMPT = ""  #@param {type:"string"}
# 
# if not TILES_PROMPT
#   TILES_PROMPT = PROMPT
# 
# tiled_prompts = []
# for x in range(TILES_WIDTH):
#   for y in range(TILES_HEIGHT):
#     tiled_prompts.append(PROMPTS)
# PROMPTS = tiled_prompts